# Capstone proposal by Valentin

# CRM Predictions

# 4 - Machine Learning

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

In [2]:
# Create a mapping of segment labels to integer values
class_map = {
    'VIP': 4,
    'Good Customers': 3,
    'Sleeping Beauty': 2,
    'Low Value Customers': 1,
    'Sleeping Dog': 0,
}

class_labels = [0, 1, 2, 3, 4]
class_names = ['Sleeping Dog', 'Low Value Customers', 'Sleeping Beauty', 'Good Customers', 'VIP']
colors = ['red', 'orange', 'purple', 'blue', 'green']

##### Loading dataset

In [3]:
df = pd.read_pickle('./datasubset.pkl')

## 4) Machine learning 

### (a) Phrase your project goal as a clear machine learning question

#### i) Intended outcome
> What is your intended outcome in machine learning terms?

The intended outcome is to build a __predictive model that can accurately predict future customer classes based on the current information available about the customers__. This is a __multiclass classification__ problem, where the goal is to assign each customer to one of the predefined classes. The model's performance will be evaluated using appropriate metrics, such as weighted F1 score, which combines both precision and recall and accounts for class imbalance.

One important consideration is to find the right balance between model complexity and generalization. While it is desirable to achieve high precision and recall on the training data, it is essential for the model to generalize well outside the training data to perform well on new, unseen data. The main objective of the project is to enable informed decisions based on the predicted customer classes, helping marketing efforts.

#### ii) Features and target variable
> What are the features and the target variable that you are using?

Our target variable is the customer classes in one year, encoded as integers according to the following mapping:

```python
class_labels = [0, 1, 2, 3, 4]
class_names = ['Sleeping Dog', 'Low Value Customers', 'Sleeping Beauty', 'Good Customers', 'VIP']
```


For the features, we will use all the available variables provided by the `get_preprocessed_pickles` function. These features include demographic information, transaction history, and other relevant customer attributes.

In [4]:
def get_preprocessed_pickles(df):
    """
    Preprocesses df by 
    - applying one-hot encoding to categorical features
    - converting dependent variable to integer based on a predefined mapping
    - moves dependent variable to last column 
    - splits dataframe into train and test sets
    - remove outliers from X_train and y_train
    - saves resulting train set, test set, and column names as pickle files
    """
    df = df.copy()
    # Convert target to integer based on mapping
    df['target_label'] = df['target_label'].map(class_map)
    
    # Apply one-hot encoding to categorical features
    df_encoded = pd.get_dummies(df)
    # Move target label column to be last column in df
    cols = list(df_encoded.columns)
    cols.remove('target_label')
    cols.append('target_label')
    df_encoded = df_encoded[cols]
    
    # Split dataframe into train and test sets
    X = df_encoded.iloc[:, :-1].values
    y = df_encoded.iloc[:, -1].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    # Remove outliers from X_train and y_train
    outlier_col_name = 'outlier'
    outlier_col_idx = df_encoded.columns.get_loc(outlier_col_name)
    rows_to_remove = np.where(X_train[:, outlier_col_idx] == 1)[0]
    X_train = np.delete(X_train, rows_to_remove, axis=0)
    y_train = np.delete(y_train, rows_to_remove, axis=0)
    
    # Remove column 'outlier' from X_train, X_test and df_encoded
    df_encoded = df_encoded.drop(outlier_col_name, axis=1)
    X_train = np.delete(X_train, outlier_col_idx, axis=1)
    X_test = np.delete(X_test, outlier_col_idx, axis=1)    
    
    # Save test set as a pickle file
    testset = {
        'X_test': X_test,
        'y_test': y_test,
        'random_state': 0
    }
    with open('testset.pkl', 'wb') as f:
        pickle.dump(testset, f)

    # Save train set as a pickle file
    trainset = {
        'X_train': X_train,
        'y_train': y_train,
        'random_state': 0
    }
    with open('trainset.pkl', 'wb') as f:
        pickle.dump(trainset, f)

    # Save column names
    with open('columns.pkl', 'wb') as f:
        pickle.dump(df_encoded.columns, f)

In [5]:
get_preprocessed_pickles(df)

Before fitting machine learning models, we plan to explore the data using Principal Component Analysis (PCA). By visualizing the first two principal components, we can assess whether the model will need to capture linear or non-linear relationships between the features and the target variable.

Additionally, we will incorporate PCA as an optional part of the pipeline in some of our models. This will allow us to reduce the dimensionality of the feature space while retaining the most important variance in the data. We will also use feature importance scores from models like Random Forest to prune less important features (iteratively) and reduce the number of features going into the modeling process while trying to maintain the same performance level. The goal here is to computation speed.

#### iii) Regression or classification ? 
> Is your question a regression or a classification problem?

My problem is a classification problem.

### (b) What models are you planning to use and why?

##### 1) K-Nearest Neighbors (KNN) classifier
Model interpretability, suitability, scalability, diversity, …?
- __Interpretability__ : KNN is a simple and intuitive model that is easy to understand. It predicts the class of a new data point based on the majority class among its K nearest neighbors in the training data. 
- __Suitability/Scalability__: The problem in our case is that our dataset has a relatively high number of features and KNN suffers from the curse of dimensionality, meaning in higher dimensions, all points tend to be equidistants. KNN should perform better computation wise than SVM, but we will try different algorithms to increase computation.
- __Diversity/Robustness__: KNN can be prone to overfitting depending on number of neighbors. The optimization of our KNN classifier has to be mindful of overfitting as outliers or noise can be reduced in influence by increase number of neighbors.

Methodologies to be used to train and finetune my models?

- We will use scikit-learn's GridSearchCV to find the best hyperparameters. The grid search involves an exhaustive search through a specified parameter grid and evaluates the model with cross-validation to find the best combination of hyperparameters. It will return both validation score and train score with return_train_score=True and this can be use to assess overfitting, as well as score on unseen test data. The grid search might end up looking similar to 

```python
grid_search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=3,
    n_jobs=6,
    verbose=1,
    return_train_score=True,
    scoring={
        'f1': make_scorer(f1_score, average='weighted'),
        'precision': make_scorer(precision_score, average='weighted'),
        'recall': make_scorer(recall_score, average='weighted')
    },
    refit='f1'  
)
```

Model Parameters
- __n_neighbors__: This is the number of neighbors (K) that KNN will consider when making a prediction. We will experiment with a range of odd values to avoid ties. A small value of K will make the model sensitive to noise and more flexible, while a larger value will make it more stable and robust to noise but less flexible. Intuitively, we would want to avoid n-neighbors = 1 as this would imply high overfitting.
- __algorithm__: We will use the default 'auto' setting, which selects the most appropriate algorithm based on the dataset ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)). And we will also try out 'brute', and keep we 'brute' if the results are similar as computation speed should be better with 'brute'.
- __distance__ : We will try different weights, distance and uniform. By setting weight='distance' the model weights the target values of the nearest neighbors by their inverse distance. This creates a continuous spectrum of predictions, unlike weight='uniform' which creates regions of constant predictions (from [course](https://learn.extensionschool.ch/learn/programs/applied-data-science-machine-learning-v3/subjects/decision-trees-and-svms-v4/units/different-model-families-v2)).
- __Scaler__: We may also integrate different scalers into our pipeline, such as StandardScaler, and RobustScaler. Scaling the features is important because KNN is a distance-based algorithm, and features on different scales can negatively impact the model's performance. We try
    - 'passthrough': No scaling is applied to the data. We have seen that sometimes no scaler performs simply better than scaling.
    - StandardScaler(): Assumes that the data is normally distributed and scales the features to have a mean of 0 and a standard deviation of 1. 
    - RobustScaler(): Scales the features using interquartile range. It is good when data is not normally distributed.
- __PCA__: We will try out PCA to reduce dimensionality of our dataset, as it helps for dimensionality reduction by reducing the number of features by transforming the original features into a new set of components. Maybe this could help with KNN as it would lessen curse of dimensionality.

##### 2) Logistic regression
Model interpretability, suitability, scalability, diversity, …?
- __Interpretability__: Logistic Regression is a relatively simple and interpretable model. The model coefficients can provide insights into the relationship between the features and the target variable (it is the classification equivalent of Linear Regression).
- __Suitability__: Logistic Regression is well-suited for multiclass classification tasks. Their linear treatment of all features allows them to extrapolate beyond the values seen during training (from [course](https://learn.extensionschool.ch/learn/programs/applied-data-science-machine-learning-v3/subjects/decision-trees-and-svms-v4/units/different-model-families-v2))
- __Scalability__: From initial tests, we have seen that Logistic regression scales relatively well with our dataset size.
- __Diversity/Robustness__: By incorporating regularization, Logistic Regression can be made more robust to outliers and noise. Still Logistic Regression would benefit from feature engineering that increase how normally distributed our features are.

Methodologies to be used to train and finetune my models?
- Also GridSearchCV

Model Parameters
- We will try both the one-vs.-rest (OvR) strategy and the softmax regression model. OvR with __solver__ liblinear and Softmax or Multinomial with solver saga (Stochastic Average Gradient Descent with an Adaptive learning rate). [Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
- __$C$ parameter__ in both cases, $C$ is the inverse of the regularization strength (smaller values indicate stronger regularization). Regularization helps in fitting overfitting as it penalizes stronger coefficients.
- We will try both __penalties l1 and l2__ : the penalty hyperparameter determines the type of regularization applied to the model. l1 indicates L1 regularization (Lasso) and l2 indicates L2 regularization (Ridge). L1 regularization tends to produce sparse models, while L2 regularization encourages the model to distribute weights more evenly across features. 
- __Scaler__: We may also integrate different scalers into our pipeline, same as KNN.
- __PCA__: We will try with and without PCA to reduce dimensionality of our dataset.

##### 3) Random forest
Model interpretability, suitability, scalability, diversity, …?
- __Interpretability__: Random Forest is relatively easy to interpret compared to deep learning models, as it is an ensemble of decision trees. Each decision tree can be visualized and we will start with a Decision Tree with depth of three to visualize which variables create first branches. Additionally, feature importances can be extracted, which we will consider as an iterative step in our process to remove some variables from our dataset and increase training speed of our models (if we don't dramatically decrease weighted f1 of course). Random Forest is prone to overfitting, but less than decision trees of course, as it is an average of decision trees.
- __Scalability__: From what we have seen, Random Forest scales relatively well, but one has to keep in mind to not test to many features combination. This part of the [course](https://learn.extensionschool.ch/learn/programs/applied-data-science-machine-learning-v3/subjects/decision-trees-and-svms-v4/units/different-model-families-v2) is to keep in mind.
> If model is unconstrained in depth we get overfitting and the model predicts exactly the target values of samples in the training set. One can counteract such overfitting by fixing the pruning parameters like max_depth, min_samples_split, or min_samples_leaf. For random forests, the above points hold as well but the last point is mitigated by the averaging across multiple trees using different feature and sample spaces. In general, models that create their predictions by averaging or aggregating the target values of subsets of the training set, can not go beyond what they have been exposed to in training. Thus these models can only predict within the range of target values observed in training. Unlike linear models, they cannot extrapolate beyond the training data. This is the case for decision trees and random forests since the predictions are calculated from the mean target value for each leaf.
- __Diversity / Robustness__: The Random Forest model introduces diversity among the individual trees by using a random subset of features at each split and bootstrapped samples for training (with default setting bootstrap=true, [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)). Random Forests are robust to outliers and noise in the data since the decision trees' averaging process reduces the impact of any single tree's errors. 

Methodologies to be used to train and finetune my models?

- Also GridSearchCV

Model Parameters
- __max_depth__ is the maximum depth of each tree in the Random Forest. It determines the maximum number of nodes from the root to the farthest leaf.
- __n_estimators__ is the number of trees in the Random Forest. The more trees, the more robust the model becomes, but it can also increase training time.
- __criterion__ determines the impurity measure used to split nodes in the decision trees. We will use Gini impurity, which indicates the likelihood of new, random data being misclassified if it were given a random class label according to the class distribution in the dataset [source](https://www.learndatasci.com/glossary/gini-impurity/). We will try other parameters like entropy, but will end up using Gini if it perfoms similarly as it is more straighforward.
- __min_samples_split__ or __min_samples_leaf__ . Here point from course stands and we will have to investigate the tradeoff between overfitting and test accuracy. These parameters should help fight overfitting, the question of what would be tradeoff with accuracy on validation.

##### 4) Neural network

Model interpretability, suitability, scalability, diversity, …?

- __Flexibility/Scalability__: Neural networks are highly flexible and can model complex, non-linear relationships. We plan on using a neural network with Dense layers, which is a suitable model for multiclass classification problems. It is chosen due to its scalability with respect to the number of layers and neurons, and the ability to learn complex patterns from the input data. We plan on reusing a model close to the dense layer part of course 4 project. 
- __Interpretability__: not the best, it tends to be hard to interprets the relationships between input features and output classes, especially with additional dense layer between input and output.

Methodologies to be used to train and finetune my models?

- Using Keras with the TensorFlow backend to build, train, and fine-tune the model. I will use GridSearchCV to find best hyperparameters for the model with using early stopping to prevent overfitting by stopping the training process when the validation accuracy stops improving.
- Sequential is the Keras model API that allows us to stack layers in a linear manner. The Dense layer is the simplest and most common type of layer, where every input node is connected to every output node with a weight.
- In our case, we only have one output node since we are doing binary classification, and we use the `softmax` activation function to convert output value between 0 and 1.
- We will use as loss function `sparse_categorical_crossentropy` as this is the case of multi-class classification where the output is a probability distribution over multiple classes.

Model Parameters
- __batch_size__ determines the number of samples to be processed in each update during training. 
- __reg_strength__ is the regularization strength for L2 regularization applied to the model's weights. A smaller value means stronger regularization. We will apply regularization in dense layers and try no regularization or stronger values.
- __dropout__ is the fraction of the input units to drop at each update during training. This helps prevent overfitting. We will have dropout tried in dense layers to also try and fight overfitting
- __learning_rate__ controls the step size for the optimizer during training. A higher learning_rate can help to escape local minima, but too high a learning_rate can also prevent convergence.
- __optimizer__, at the start we plan on using Adam optimizer as it performed well on course 4.  
- We plan also to use as variables the __number of neurons__ in the initial layers and in the intermediary layers. I plan also to use potentially different __number of layers__, as part of grid search similar to what is done in this [article](https://datagraphi.com/blog/post/2019/12/17/how-to-find-the-optimum-number-of-hidden-layers-and-nodes-in-a-neural-network-model)

#### ii) Priorities
> A ranking of your approaches: priority, optional or “nice to add”

##### Priority 
Finding a model that perfoms well on unseen test data, with limited overfitting. We would want to maximize application in production, and the context is customer behavior, which can in itself evolve over time.

##### Optional
If possible, find a model that perfoms better on Good Customers and VIP as 
- Predicting Good Customers and VIP from existing classes as this can be directly used to argue for better incentives for those customer classes.
- Predicting well VIP going to lower value classes as this could also be maybe prevented by offering strong incentives to those VIP.

##### Nice to add
- Additional ML Models : articles such as this [one](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py) make it interesting to think about testing other classifiers if time permits to see if we get better performance. Still, as these models have not been seen in class (such as Naive Bayes), we will use caution.
- Integrating outliers removal as part of pipe on some models : this [article](https://towardsdatascience.com/custom-transformers-and-ml-data-pipelines-with-python-20ea2a7adb65) explains how to build customer transfomer and integrate it in pipeline. As outliers removal is always tricky, as thresholds have to be defined, and outliers could constitute in themselves novelties (meaning emerging customer behavior) as explained in this part of the [course](https://learn.extensionschool.ch/learn/programs/applied-data-science-machine-learning-v3/subjects/cleaning-the-data-v4/units/detecting-and-filtering-outliers-v4), it could be interesting to comparer performances of models with and without outliers.

### (c) Please tell us your detailed machine learning strategy 

#### i) Preprocessing
> Preprocessing steps of your data for each machine learning model

- For KNN and Logistic Regression, where we will integrate scaler and PCA within pipeline as part of gridsearch, preprocessing will happen within GridSearch. This below is an example of what it might look like.

```python
# Create a list of scalers
scalers = ['passthrough', StandardScaler(), RobustScaler()]

# Define the parameter grid
param_grid = {
    'preprocessor__scaler': scalers,
    'dim_reduction': ['passthrough', PCA(n_components=0.95)],
    'classifier__parameter': values,
}

# Create pipeline
pipe = Pipeline([
    ('preprocessor', ColumnTransformer(transformers=[
        ('scaler', 'passthrough', list(range(X_train.shape[1]))),
    ])),
    ('dim_reduction', 'passthrough'),
    ('classifier', ) #here either Knn or Logistic regression
])
````

- For RandomForest, we will try and integrate RobustScaler() and no scaling as part of pipeline as it is generally considered that RandomForest does not needs rescaling, as found in initial research [stackoverflow](https://stackoverflow.com/questions/8961586/do-i-need-to-normalize-or-scale-data-for-randomforest-r-package)
- For Neural Network where we are using Keras API, we will use no PCA and RobustScaler() to start with. We might also try different Scalers if time permits on both models before going outside of GridSearchCV, see if we get improve performance or not (after additional EDA and additional feature engineering).

#### iii) Baseline model

Our baseline model will best of 
- DummyClassifier(strategy='most_frequent') [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html)
- DummyClassifier(strategy='uniform')
- Another baseline which translates no changes in rfm_labels, meaning value of y_test is value of rfm_label ('VIP' stays 'VIP', 'Good Customers' stays 'Good Customers', 'Low Value Customers' stays 'Low Value Customers')

#### iv) Models evaluation
> The metrics and methodologies you are considering to evaluate and compare your models

To assess model performance, we will also look at precision and recall, which in simple terms are
- __Precision__ : (example) correct VIP over all VIPs we identify in our classification. Cost of misclassification here is that we are going to probably send promo codes or incentives to people that where not going to be VIPs, or total cost of misclassification could be calculated by multiplying the number of our False Positives times the cost of the incentive we would allow to that class.
- __Recall__ : (example) now recall measures how many VIP we identified over all potential VIPs. The cost of recall in this case is not identifying we could have valorized a customer, or in the case of another class (for instance Sleeping Dog), the number of False Negatives is the number of Sleeping Dog we would have fail to identify and prevent.

In our case, both metrics are important as we are looking to 
- be precise and sent incentivize only True Positives, as we don't want the cost of the project in production to be disproportionate
- be sensitive and maximize our opportunities, and not fail to identify opportunities.

What is why we will use in GridSearchCV and in our evaluation F1 Score as it provides a balance between both precision and recall.
As we will be displaying confusion matrix and F1 score for each rfm_labels, we will also take into account weigthed_avg. 
- macro_avg which computes average accuracy metric over all classes without taking into account the class distribution.
- weighted_avg which computes the average metric over all classes, but takes into account the class distribution.

As we can see in tables below, we have some class imbalance in for overall on target_label so we might want to use weighted_avg, otherwise our scoring is going to over represent Sleeping Beauty. This is why we will use weighted_avg

In [6]:
df.groupby(['target_label']).size()

target_label
Good Customers          8226
Low Value Customers    12269
Mono Customer            112
Sleeping Beauty        16877
Sleeping Dog           13262
VIP                    13052
dtype: int64

In [7]:
df.groupby(['rfm_label','target_label']).size()

rfm_label            target_label       
Good Customers       Good Customers          2882
                     Low Value Customers     2306
                     Mono Customer             13
                     Sleeping Beauty         3738
                     Sleeping Dog            1735
                     VIP                     2567
Low Value Customers  Good Customers          3339
                     Low Value Customers     9081
                     Mono Customer             94
                     Sleeping Beauty         9811
                     Sleeping Dog           10848
                     VIP                     1415
VIP                  Good Customers          2005
                     Low Value Customers      882
                     Mono Customer              5
                     Sleeping Beauty         3328
                     Sleeping Dog             679
                     VIP                     9070
dtype: int64

We will also look at overall confusion matrix, and confusion matrix for each rfm_labels, meaning how well future class is predicted for VIP, Good Customers and Low Value Customers. This will help indicate whether our model performs differently by rfm_labels. It also gives a nice visual overview.